# Saving SCF results on disk and SCF checkpoints

For longer DFT calculations it is pretty standard to run them on a cluster
in advance and to perform postprocessing (band structure calculation,
plotting of density, etc.) at a later point and potentially on a different
machine.

To support such workflows DFTK offers the two functions `save_scfres`
and `load_scfres`, which allow to save the data structure returned
by `self_consistent_field` on disk or retrieve it back into memory,
respectively. For this purpose DFTK uses the
[JLD2.jl](https://github.com/JuliaIO/JLD2.jl) file format and Julia package.

> **Availability of `load_scfres`, `save_scfres` and checkpointing**
>
> As JLD2 is an optional dependency of DFTK these three functions are only
> available once one has *both* imported DFTK and JLD2 (`using DFTK`
> and `using JLD2`).

To illustrate the use of the functions in practice we will compute
the total energy of the O₂ molecule at PBE level. To get the triplet
ground state we use a collinear spin polarisation
(see Collinear spin and magnetic systems for details)
and a bit of temperature to ease convergence:

In [1]:
using DFTK
using LinearAlgebra
using JLD2
using PseudoPotentialData

d = 2.079  # oxygen-oxygen bondlength
a = 9.0    # size of the simulation box
lattice = a * I(3)
pseudopotentials = PseudoFamily("cp2k.nc.sr.pbe.v0_1.semicore.gth")
O = ElementPsp(:O, pseudopotentials)
atoms     = [O, O]
positions = d / 2a * [[0, 0, 1], [0, 0, -1]]
magnetic_moments = [1., 1.]

Ecut  = 10  # Far too small to be converged
model = model_DFT(lattice, atoms, positions;
                  functionals=PBE(),
                  temperature=0.02, smearing=Smearing.Gaussian(),
                  magnetic_moments)
basis = PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])

scfres = self_consistent_field(basis, tol=1e-2, ρ=guess_density(basis, magnetic_moments))
save_scfres("scfres.jld2", scfres);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -27.64800912096                   -0.13    0.001    7.5    426ms
  2   -28.92456851650        0.11       -0.83    0.688    2.0    629ms
  3   -28.93111803647       -2.18       -1.13    1.192    2.0    110ms
  4   -28.93774962933       -2.18       -1.19    1.776    1.5   84.2ms
  5   -28.93924836688       -2.82       -1.06    2.000    2.0   91.0ms
  6   -28.93957812528       -3.48       -1.87    1.975    1.5   81.3ms
  7   -28.93961156694       -4.48       -2.86    1.984    1.0   81.3ms


In [2]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             16.7728512
    AtomicLocal         -58.4960399
    AtomicNonlocal      4.7090966 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3618046
    Xc                  -6.3913832
    Entropy             -0.0008897

    total               -28.939611566936

The `scfres.jld2` file could now be transferred to a different computer,
Where one could fire up a REPL to inspect the results of the above
calculation:

In [3]:
using DFTK
using JLD2
loaded = load_scfres("scfres.jld2")
propertynames(loaded)

(:α, :history_Δρ, :converged, :occupation, :occupation_threshold, :algorithm, :basis, :runtime_ns, :n_iter, :n_matvec, :history_Etot, :εF, :energies, :ρ, :timedout, :n_bands_converge, :τ, :eigenvalues, :ψ, :ham)

In [4]:
loaded.energies

Energy breakdown (in Ha):
    Kinetic             16.7728512
    AtomicLocal         -58.4960399
    AtomicNonlocal      4.7090966 
    Ewald               -4.8994689
    PspCorrection       0.0044178 
    Hartree             19.3618046
    Xc                  -6.3913832
    Entropy             -0.0008897

    total               -28.939611566936

Since the loaded data contains exactly the same data as the `scfres` returned by the
SCF calculation one could use it to plot a band structure, e.g.
`plot_bandstructure(load_scfres("scfres.jld2"))` directly from the stored data.

Notice that both `load_scfres` and `save_scfres` work by transferring all data
to/from the master process, which performs the IO operations without parallelisation.
Since this can become slow, both functions support optional arguments to speed up
the processing. An overview:
- `save_scfres("scfres.jld2", scfres; save_ψ=false)` avoids saving
  the Bloch wave, which is usually faster and saves storage space.
- `load_scfres("scfres.jld2", basis)` avoids reconstructing the basis from the file,
  but uses the passed basis instead. This save the time of constructing the basis
  twice and allows to specify parallelisation options (via the passed basis). Usually
  this is useful for continuing a calculation on a supercomputer or cluster.

See also the discussion on Input and output formats on JLD2 files.

## Checkpointing of SCF calculations
A related feature, which is very useful especially for longer calculations with DFTK
is automatic checkpointing, where the state of the SCF is periodically written to disk.
The advantage is that in case the calculation errors or gets aborted due
to overrunning the walltime limit one does not need to start from scratch,
but can continue the calculation from the last checkpoint.

The easiest way to enable checkpointing is to use the `kwargs_scf_checkpoints`
function, which does two things. (1) It sets up checkpointing using the
`ScfSaveCheckpoints` callback and (2) if a checkpoint file is detected,
the stored density is used to continue the calculation instead of the usual
atomic-orbital based guess. In practice this is done by modifying the keyword arguments
passed to # `self_consistent_field` appropriately, e.g. by using the density
or orbitals from the checkpoint file. For example:

In [5]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-2, checkpointargs...);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ----   ------
  1   -27.64750293488                   -0.13    0.001   0.80    7.0    246ms
  2   -28.92453454523        0.11       -0.83    0.685   0.80    2.0    879ms
  3   -28.93108291898       -2.18       -1.13    1.188   0.80    2.0    113ms
  4   -28.93772996838       -2.18       -1.18    1.775   0.80    1.5   89.3ms
  5   -28.93929943042       -2.80       -1.09    2.000   0.80    1.5   90.5ms
  6   -28.93958281278       -3.55       -1.89    1.976   0.80    1.5    122ms
  7   -28.93961188486       -4.54       -3.00    1.985   0.80    1.0   89.6ms


Notice that the `ρ` argument is now passed to kwargs_scf_checkpoints instead.
If we run in the same folder the SCF again (here using a tighter tolerance),
the calculation just continues.

In [6]:
checkpointargs = kwargs_scf_checkpoints(basis; ρ=guess_density(basis, magnetic_moments))
scfres = self_consistent_field(basis; tol=1e-3, checkpointargs...);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   α      Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ----   ------
  1   -28.93961282874                   -3.34    1.985   0.80   10.5    190ms


Since only the density is stored in a checkpoint
(and not the Bloch waves), the first step needs a slightly elevated number
of diagonalizations. Notice, that reconstructing the `checkpointargs` in this second
call is important as the `checkpointargs` now contain different data,
such that the SCF continues from the checkpoint.
By default checkpoint is saved in the file `dftk_scf_checkpoint.jld2`, which can be changed
using the `filename` keyword argument of `kwargs_scf_checkpoints`. Note that the
file is not deleted by DFTK, so it is your responsibility to clean it up. Further note
that warnings or errors will arise if you try to use a checkpoint, which is incompatible
with your calculation.

We can also inspect the checkpoint file manually using the `load_scfres` function
and use it manually to continue the calculation:

In [7]:
oldstate = load_scfres("dftk_scf_checkpoint.jld2")
scfres   = self_consistent_field(oldstate.basis, ρ=oldstate.ρ, ψ=oldstate.ψ, tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Magnet   Diag   Δtime
---   ---------------   ---------   ---------   ------   ----   ------
  1   -28.93866808195                   -2.32    1.985    5.5    123ms
  2   -28.93945065159       -3.11       -2.78    1.985    1.0   87.0ms
  3   -28.93961082794       -3.80       -2.60    1.985    4.5    167ms
  4   -28.93961209896       -5.90       -2.74    1.985    1.0   76.8ms
  5   -28.93961268109       -6.23       -2.89    1.985    1.0   72.3ms
  6   -28.93961285717       -6.75       -3.00    1.985    1.0   75.3ms
  7   -28.93961307921       -6.65       -3.28    1.985    1.5   79.0ms
  8   -28.93961312238       -7.36       -3.42    1.985    1.0   73.4ms
  9   -28.93961316530       -7.37       -3.87    1.985    2.0   84.3ms
 10   -28.93961317227       -8.16       -5.14    1.985    2.0   84.4ms


Some details on what happens under the hood in this mechanism: When using the
`kwargs_scf_checkpoints` function, the `ScfSaveCheckpoints` callback is employed
during the SCF, which causes the density to be stored to the JLD2 file in every iteration.
When reading the file, the `kwargs_scf_checkpoints` transparently patches away the `ψ`
and `ρ` keyword arguments and replaces them by the data obtained from the file.
For more details on using callbacks with DFTK's `self_consistent_field` function
see Monitoring self-consistent field calculations.

(Cleanup files generated by this notebook)

In [8]:
rm("dftk_scf_checkpoint.jld2")
rm("scfres.jld2")